# Más `pandas`

## Manejando datos faltantes

Hemos visto que `pandas` automágicamente es capaz de manejar valores faltantes o inexistentes, a través de distintas etiquetas como `NaN`, `NA`, etc., dependiendo del tipo de dato que se esté utilizando. Más allá de la lectura de los datos, muchos de los métodos nativos de `pandas` son capaces de trabajar aún cuando falten datos. Sin embargo, puede ser necesario intervenir efectivamente sobre esos datos para poder continuar con ese procesamiento.

In [1]:
import numpy as np 
import pandas as pd

In [2]:
d = np.array([['auto','moto',np.nan,None,'bicicleta'],[4,2,0.0,None,2]])
d


array([['auto', 'moto', nan, None, 'bicicleta'],
       [4, 2, 0.0, None, 2]], dtype=object)

In [3]:
s = pd.DataFrame(d.T,columns=['vehículos','ruedas'])
s

,vehículos,ruedas
0,auto,4
1,moto,2
2,NaN,0.0
3,None,None
4,bicicleta,2


In [4]:
s.isna()


,vehículos,ruedas
0,False,False
1,False,False
2,True,False
3,True,True
4,False,False


In [5]:
s.dropna() # equivalente a s[s.notna()]

,vehículos,ruedas
0,auto,4
1,moto,2
4,bicicleta,2


In [6]:
s.dropna(axis=1) # equivalente a s.dropna(axis='columns')

""
0
1
2
3
4


In [7]:
s.dropna(how='all') # elimina las filas que tengan todos los valores nulos

,vehículos,ruedas
0,auto,4
1,moto,2
2,NaN,0.0
4,bicicleta,2


También se puede usar el argumento 'thresh=<valor>' para acotar la cantidad de valores inexistentes que se quieren eliminar. Por ejemplo, `thresh=3` eliminará todas aquellas filas que tienen 3 o más valores faltantes.

Es posible también que uno no pueda trabajar con valores inexistentes, y tiene que cambiarlos por algún valor. Para ello está el método `fillna`.

In [8]:
df = pd.DataFrame(np.random.standard_normal((7, 3)),columns=['A','B','C'])

df.iloc[:4,2] = np.nan
df.iloc[1:3,0] = None 

df

,A,B,C
0,0.377913,-0.770393,NaN
1,NaN,-0.009935,NaN
2,NaN,0.904126,NaN
3,-0.077318,-0.022459,NaN
4,0.827397,2.037912,-0.418672
5,0.047666,-1.048824,0.248140
6,-0.943052,1.499018,0.048949


In [9]:
df.fillna(0) # rellena los valores nulos con 0

,A,B,C
0,0.377913,-0.770393,0.000000
1,0.000000,-0.009935,0.000000
2,0.000000,0.904126,0.000000
3,-0.077318,-0.022459,0.000000
4,0.827397,2.037912,-0.418672
5,0.047666,-1.048824,0.248140
6,-0.943052,1.499018,0.048949


In [10]:
df.fillna({'A':0,'C':2}) # rellena los valores nulos con 0, 1 y 2 respectivamente

,A,B,C
0,0.377913,-0.770393,2.000000
1,0.000000,-0.009935,2.000000
2,0.000000,0.904126,2.000000
3,-0.077318,-0.022459,2.000000
4,0.827397,2.037912,-0.418672
5,0.047666,-1.048824,0.248140
6,-0.943052,1.499018,0.048949


In [11]:
df.ffill() # rellena los valores nulos con el valor anterior

,A,B,C
0,0.377913,-0.770393,NaN
1,0.377913,-0.009935,NaN
2,0.377913,0.904126,NaN
3,-0.077318,-0.022459,NaN
4,0.827397,2.037912,-0.418672
5,0.047666,-1.048824,0.248140
6,-0.943052,1.499018,0.048949


In [12]:
df.ffill(axis=1) # rellena los valores nulos con el valor anterior en la misma fila

,A,B,C
0,0.377913,-0.770393,-0.770393
1,NaN,-0.009935,-0.009935
2,NaN,0.904126,0.904126
3,-0.077318,-0.022459,-0.022459
4,0.827397,2.037912,-0.418672
5,0.047666,-1.048824,0.248140
6,-0.943052,1.499018,0.048949


In [13]:
df.bfill() # rellena los valores nulos con el valor siguiente

,A,B,C
0,0.377913,-0.770393,-0.418672
1,-0.077318,-0.009935,-0.418672
2,-0.077318,0.904126,-0.418672
3,-0.077318,-0.022459,-0.418672
4,0.827397,2.037912,-0.418672
5,0.047666,-1.048824,0.248140
6,-0.943052,1.499018,0.048949


Estos métodos para reemplazar de valores inexistentes son un caso particular de un método para reemplazar valores en forma general, denominado `replace` y puede ser útil para reemplazar valores que, por alguna razón, se encuentran fuera del rango esperado de los datos (un precio negativo, una edad mayor a 120 años, etc.). Veamos cómo funciona:

In [14]:
p = pd.Series([23,4,-8,12,27,-9])
p

0    23
1     4
2    -8
3    12
4    27
5    -9
dtype: int64

In [15]:
p.replace(-9,np.nan) # reemplaza -9 por NaN

0    23.0
1     4.0
2    -8.0
3    12.0
4    27.0
5     NaN
dtype: float64

In [16]:
p.replace({-9:np.nan,-8:0}) # reemplaza -9 por NaN y 23 por 0

0    23.0
1     4.0
2     0.0
3    12.0
4    27.0
5     NaN
dtype: float64

In [17]:
p<0 

0    False
1    False
2     True
3    False
4    False
5     True
dtype: bool

In [18]:
p[p < 0]

2   -8
5   -9
dtype: int64

In [19]:
list(p[p < 0])

[-8, -9]

In [20]:
p.replace(list(p[p < 0]),[86,22]) # encuentro los valores < 0 y los reemplazo por 86 y 22 respectivamente

0    23
1     4
2    86
3    12
4    27
5    22
dtype: int64

> **Nota:** el método `replace()` genera un nuevo dato.

## Indicadores

Otro tipo de transformación para el modelado estadístico es convertir una variable en un
_indicador_. Si una columna en un `DataFrame` tiene k valores distintos, se derivará una matriz
o `DataFrame` con k columnas conteniendo unos y ceros, por ejemplo:

In [21]:
df = pd.DataFrame({'key': ['a','a','b','d','a','c','c'],'datos': np.random.standard_normal(7)})
df

,key,datos
0,a,-0.680732
1,a,0.021716
2,b,0.722098
3,d,-0.671279
4,a,1.547057
5,c,-0.880381
6,c,-0.633695


In [22]:
pd.get_dummies(df['key'],dtype=int) # crea variables dummy

,a,b,c,d
0,1,0,0,0
1,1,0,0,0
2,0,1,0,0
3,0,0,0,1
4,1,0,0,0
5,0,0,1,0
6,0,0,1,0


## Manejando índices múltiples

Hemos visto hasta ahora que los índices nos etiquetan cada una de las filas de un `DataFrame`. Pandas tiene la posibilidad de utilizar índices múltiples o _jerárquicos_ con el objeto de añadir dimensionalidad a las tablas. 
La implementación de esta característica consiste en utilizar tuplas como índices para etiquetar cada fila:


In [23]:
# Índices jerárquicos: ciudades, productos, años
index = [
    ("Buenos Aires", "Zapatos", 2022),
    ("Buenos Aires", "Ropa", 2022),
    ("Buenos Aires", "Ropa", 2023),
    ("Córdoba", "Zapatos", 2023),
    ("Córdoba", "Ropa", 2023),
    ("Rosario", "Zapatos", 2023),
]

In [24]:
index = pd.MultiIndex.from_tuples(index, names=["Ciudad", "Producto", "Año"])
print(type(index))
index

<class 'pandas.core.indexes.multi.MultiIndex'>


MultiIndex([('Buenos Aires', 'Zapatos', 2022),
            ('Buenos Aires',    'Ropa', 2022),
            ('Buenos Aires',    'Ropa', 2023),
            (     'Córdoba', 'Zapatos', 2023),
            (     'Córdoba',    'Ropa', 2023),
            (     'Rosario', 'Zapatos', 2023)],
           names=['Ciudad', 'Producto', 'Año'])

In [25]:
# Datos
data = {
    "Ventas": [200, 150, 300, 400, 250, 500],
    "Costo": [120, 80, 180, 240, 150, 300]
}

In [26]:
df = pd.DataFrame(data, index=index)
df

Ventas  Costo
Ciudad       Producto Año                
Buenos Aires Zapatos  2022     200    120
             Ropa     2022     150     80
                      2023     300    180
Córdoba      Zapatos  2023     400    240
             Ropa     2023     250    150
Rosario      Zapatos  2023     500    300

Si queremos obtener ciertas filas específicas, usamos `.loc`. 

In [27]:
# Acceso por niveles del índice
print("\nDatos de 'Buenos Aires' en 2022:")
print(df.loc[("Buenos Aires", slice(None), 2022), :])


Datos de 'Buenos Aires' en 2022:
                            Ventas  Costo
Ciudad       Producto Año                
Buenos Aires Zapatos  2022     200    120
             Ropa     2022     150     80


La función `slice` se usa para determinar el rango de filas en cada componente del índice. `slice(None)` implica usar todos los valores posibles para dicha componente del índice.

### Agrupando

La potencia de los índices múltiples radica en poder agrupar datos de acuerdo a una determinada componente del índice. Para ello se utiliza el método `.groupby()`, que agrupa los valores de acuerdo al nivel (`level`) indicado: 

In [28]:
# Resumen por nivel del índice
print("\nVentas totales por ciudad:")
print(df.groupby(level="Ciudad")["Ventas"].sum())


Ventas totales por ciudad:
Ciudad
Buenos Aires    650
Córdoba         650
Rosario         500
Name: Ventas, dtype: int64


Si uno quisiera calcular el monto total de ventas por ciudad y por año, por ejemplo, se podría hacer:

In [29]:
# Agrupar por 'Ciudad' y 'Año' y sumar el costo
df["Total"] = df["Ventas"] * df["Costo"]
df


Ventas  Costo   Total
Ciudad       Producto Año                        
Buenos Aires Zapatos  2022     200    120   24000
             Ropa     2022     150     80   12000
                      2023     300    180   54000
Córdoba      Zapatos  2023     400    240   96000
             Ropa     2023     250    150   37500
Rosario      Zapatos  2023     500    300  150000

In [30]:
costo_anual_ciudad_x_año = df.groupby(level=["Ciudad", "Año"])["Total"].sum()

# Mostrar el resultado
print("Costo anual por ciudad por año:")
print(costo_anual_ciudad_x_año)


Costo anual por ciudad por año:
Ciudad        Año 
Buenos Aires  2022     36000
              2023     54000
Córdoba       2023    133500
Rosario       2023    150000
Name: Total, dtype: int64


In [31]:
costo_anual_ciudad = df.groupby(level=["Ciudad"])["Total"].sum()

# Mostrar el resultado
print("Costo anual por ciudad:")
print(costo_anual_ciudad)

Costo anual por ciudad:
Ciudad
Buenos Aires     90000
Córdoba         133500
Rosario         150000
Name: Total, dtype: int64


## Apilando y desapilando

Otra operación es apilar o desapilar el dataframe de índices múltiples:

In [32]:
df.unstack() # desapila el índice

Ventas         Costo           Total          
Año                     2022   2023   2022   2023     2022      2023
Ciudad       Producto                                               
Buenos Aires Ropa      150.0  300.0   80.0  180.0  12000.0   54000.0
             Zapatos   200.0    NaN  120.0    NaN  24000.0       NaN
Córdoba      Ropa        NaN  250.0    NaN  150.0      NaN   37500.0
             Zapatos     NaN  400.0    NaN  240.0      NaN   96000.0
Rosario      Zapatos     NaN  500.0    NaN  300.0      NaN  150000.0

In [33]:
df.unstack().columns

MultiIndex([('Ventas', 2022),
            ('Ventas', 2023),
            ( 'Costo', 2022),
            ( 'Costo', 2023),
            ( 'Total', 2022),
            ( 'Total', 2023)],
           names=[None, 'Año'])

Tal como se ve en el ejemplo anterior, **las columnas también pueden ser descriptas con índices jerárquicos**

In [34]:
df.unstack().unstack() # desapila el índice

Ventas                        Costo                         \
Año            2022           2023          2022           2023           
Producto       Ropa Zapatos   Ropa Zapatos  Ropa Zapatos   Ropa Zapatos   
Ciudad                                                                    
Buenos Aires  150.0   200.0  300.0     NaN  80.0   120.0  180.0     NaN   
Córdoba         NaN     NaN  250.0   400.0   NaN     NaN  150.0   240.0   
Rosario         NaN     NaN    NaN   500.0   NaN     NaN    NaN   300.0   

                Total                              
Año              2022              2023            
Producto         Ropa  Zapatos     Ropa   Zapatos  
Ciudad                                             
Buenos Aires  12000.0  24000.0  54000.0       NaN  
Córdoba           NaN      NaN  37500.0   96000.0  
Rosario           NaN      NaN      NaN  150000.0

## Categorías

Es muy posible que en nuestro conjunto de datos tengamos valores repetidos. 


In [35]:
s = pd.Series(["Pequeño", "Mediano", "Grande", "Pequeño", "Grande", "Mediano"])
s

0    Pequeño
1    Mediano
2     Grande
3    Pequeño
4     Grande
5    Mediano
dtype: object

In [36]:
s_cat = s.astype("category")
s_cat

0    Pequeño
1    Mediano
2     Grande
3    Pequeño
4     Grande
5    Mediano
dtype: category
Categories (3, object): ['Grande', 'Mediano', 'Pequeño']

In [37]:
s_cat.cat.categories


Index(['Grande', 'Mediano', 'Pequeño'], dtype='object')

In [38]:
s_cat.cat.codes

0    2
1    1
2    0
3    2
4    0
5    1
dtype: int8

In [39]:
dict(enumerate(s_cat.cat.categories))

{0: 'Grande', 1: 'Mediano', 2: 'Pequeño'}

En el caso de un `DataFrame`, uno puede convertir una columna en una categoría reasignandola:

In [40]:
precios = [100, 200, 300, 150, 250, 180]

df = pd.DataFrame({'precios':precios,'tamaño':s})
df

,precios,tamaño
0,100,Pequeño
1,200,Mediano
2,300,Grande
3,150,Pequeño
4,250,Grande
5,180,Mediano


In [41]:
df['tamaño']

0    Pequeño
1    Mediano
2     Grande
3    Pequeño
4     Grande
5    Mediano
Name: tamaño, dtype: object

In [42]:
df['tamaño'] = df['tamaño'].astype('category')
df['tamaño']

0    Pequeño
1    Mediano
2     Grande
3    Pequeño
4     Grande
5    Mediano
Name: tamaño, dtype: category
Categories (3, object): ['Grande', 'Mediano', 'Pequeño']

El beneficio principal del uso de categorías tiene que ver con la eficiencia en la memoria y en las operaciones:

In [43]:
N = 10_000_000

s = pd.Series(['a','b','c','d']* (N//4))
s_cat = s.astype('category')

In [44]:
print(s.memory_usage(deep=True))
print(s_cat.memory_usage(deep=True))

500000132
10000504


In [45]:
%timeit s.value_counts()

363 ms ± 15.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [46]:
%timeit s_cat.value_counts()

38.9 ms ± 2.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


-----

## Ejercicio 15 (a)


1. Retome el `DataFrame` creado en el ejercicio 14(a), y genere un nuevo `DataFrame` utilizando adecuadamente índices jerárquicos.


-----